In [ ]:
###########################
# Импорты и инициализации #
###########################

import cv2
import os
import glob
import numpy as np
from matplotlib import pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Flatten, Activation
from keras.layers import Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.utils import np_utilsscx
from keras.preprocessing import image

from keras.datasets import cifar10

# Задаем seed для повторяемости результатов
np.random.seed(42)

# image_width = 168
# image_height = 168

In [ ]:
#############################################
# Масштабируем изображения к одному размеру #
#############################################

for (dirpath, dirnames, filenames) in os.walk('train_data'):
    for filename in filenames:
        if filename.endswith('.bmp'):
            img = cv2.imread(os.sep.join([dirpath, filename]))
            res = cv2.resize(img, (32, 32), interpolation = cv2.INTER_CUBIC)
            cv2.imwrite(os.sep.join([dirpath, filename[5:]]), res)

In [ ]:
###################################################
# Считываем данные и подготавливаем их к обучению #
###################################################

list_of_files = []
target = []
i = -1
mapa = {}
for (dirpath, dirnames, filenames) in os.walk('Dataset_new/train_data'):
    for filename in filenames:
        if filename.endswith('.bmp'):
            target.append(i)
            list_of_files.append(os.sep.join([dirpath, filename]))
    mapa[i] = dirpath
    i += 1
print(mapa)
data = []
flag = True
for file_name in list_of_files:
#     data.append(cv2.imread(file_name))
    img = image.load_img(file_name, target_size = (128, 128), grayscale = True)
#     t = img[72:240, 40:208]
    tmp = image.img_to_array(img)
#     t = tmp[72:240, 40:208]
    tmp /= 255
    data.append(tmp)
#     if (flag):
#         train_data = np.array(tmp)
#         flag = False
#     else:
#         np.append(train_data, tmp, axis = 1)
#         print(train_data.shape)
#     train_data.append(tmp)
#     np.append(train_data_2, tmp, axis = 0)

train_data = np.array(data)
# for i, item in enumerate(train_data):
#     t = item[72:240, 40:208]
#     train_data[i] = t
train_target = np_utils.to_categorical(target, 4)
# train_target.reshape((-1, 1))

In [ ]:
##############################
# Создаем и обучаем модель 1 #
##############################

# Создаем модель
model = Sequential()

# ПЕРВЫЙ КАСКАД СЕТИ
# Первый сверточный слой
model.add(Convolution2D(32, 5, 5,
                 padding = 'same',
                 input_shape = (1, 60, 77),
                 activation = 'relu'))
# Второй сверточный слой
model.add(Convolution2D(32, 5, 5, activation = 'relu'))
# Слой подвыборки
model.add(MaxPooling2D(pool_size = (2, 2), dim_ordering="tf"))
# Слой регуляризации
model.add(Dropout(0.25))
# ВТОРОЙ КАСКАД СЕТИ
# Третий сверточный слой
model.add(Convolution2D(64, 5, 5,
                 padding = 'same',
                 activation = 'relu'))
# Четвертый сверточный слой
model.add(Convolution2D(64, 5, 5, activation = 'relu'))
# Второй слой подвыборки
model.add(MaxPooling2D(pool_size = (2, 2), dim_ordering="tf"))
# Слой регуляризации
model.add(Dropout(0.25))

# КЛАССИФИКАТОР
# Преобразование из двумерного вида в плоский
model.add(Flatten())
# Полносвязный слой
model.add(Dense(512, activation = 'relu'))
# Слой регуляризации
model.add(Dropout(0.5))
# Выходной слой
model.add(Dense(4, activation = 'softmax'))

# КОМПИЛЯЦИЯ СЕТИ
model.compile(loss = 'categorical_crossentropy',
              optimizer = 'SGD',
              metrics = ['accuracy'])

# ОБУЧЕНИЕ СЕТИ
model.fit(train_data, train_target,
          batch_size = 32,
          epochs = 25,
          validation_split = 0.1,
          shuffle = True)

In [ ]:
##############################
# Создаем и обучаем модель 2 #
##############################

# Создаем последовательную сеть
model = Sequential()

# КАСКАД СВЕРТОК
# Первый сверточный слой
model.add(Convolution2D(75, 5, 5,
                 padding = 'valid',
                 input_shape = (1, 32, 32)))
model.add(Activation('relu'))
# Слой подвыборки
model.add(MaxPooling2D(pool_size = (2, 2)))
# Слой регуляризации
model.add(Dropout(0.2))
# Второй сверточный слой
model.add(Convolution2D(100, 5, 5,
                 activation = 'relu'))
model.add(Activation('relu'))
# Второй слой подвыборки
model.add(MaxPooling2D(pool_size = (2, 2), dim_ordering="tf"))
# Слой регуляризации
model.add(Dropout(0.2))

# КЛАССИФИКАТОР
# Преобразование из двумерного вида в плоский
model.add(Flatten())
# Полносвязный слой
model.add(Dense(500))
model.add(Activation('relu'))
# Слой регуляризации
model.add(Dropout(0.5))
# Выходной слой
model.add(Dense(4))
model.add(Activation('softmax'))

# КОМПИЛЯЦИЯ СЕТИ
model.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

print(model.summary())

# ОБУЧЕНИЕ СЕТИ
model.fit(train_data, train_target,
          batch_size = 32,
          epochs = 25,
          validation_split = 0.2,
          verbose = 1)

In [ ]:
###################
# Сохранение сети #
###################

# model_json = model.to_json()
# json_file = open('model.json', 'w')
# json_file.write(model_json)
# json_file.close()
# model.save_weights('model.h5')

model.save('model.h5')

In [ ]:
#################
# Загрузка сети #
#################

from keras.models import load_model, model_from_json

# json_file = open('model.json', 'r')
# loadedmodel_json = json_file.read()
# json_file.close()
# loaded_model = model_from_json(loaded_model_json)
# loaded_model.load_weight('model.h5')

model = load_model('model.h5')

In [ ]:
#####################################################
# Подготовка тестовых данных и проверка сети на них #
#####################################################

list_of_files = []
target = []
i = -1
mapa = {}
for (dirpath, dirnames, filenames) in os.walk('Dataset_new/test_data/test_data'):
    for filename in filenames:
        if filename.endswith('.bmp'):
            target.append(i)
            list_of_files.append(os.sep.join([dirpath, filename]))
    mapa[i] = dirpath
    i += 1
print(mapa)

data = []
for file_name in list_of_files:
    img = image.load_img(file_name, target_size = (128, 128), grayscale = True)
    tmp = image.img_to_array(img)
    tmp /= 255
    data.append(tmp)

test_data = np.array(data)
test_target = np_utils.to_categorical(target, 4)

# Проверяем точность модели на тестовых данных
scores = model.evaluate(test_data, test_target, verbose = 0)

# Печатаем точность
print("Точность работы на тестовых данных: %.2f%%" %(scores[1] * 100))

In [ ]:
##############################
# Создаем и обучаем модель 3 #
##############################

# Создаем модель
model = Sequential()

# ПЕРВЫЙ КАСКАД СЕТИ
# Первый сверточный слой
model.add(Convolution2D(32, 3, 3,
                          border_mode = 'same',
                          input_shape = (32, 32, 3),
                          activation = 'relu'))
# Второй сверточный слой
model.add(Convolution2D(32, 3, 3, activation = 'relu'))
# Слой подвыборки
model.add(MaxPooling2D(pool_size = (2, 2), dim_ordering="tf"))
# Слой регуляризации
model.add(Dropout(0.25))

# ВТОРОЙ КАСКАД СЕТИ
# Третий сверточный слой
model.add(Convolution2D(64, 3, 3,
                       border_mode = 'same',
                       activation = 'relu'))
# Четвертый сверточный слой
model.add(Convolution2D(64, 3, 3,
                        border_mode = 'same',
                        activation = 'relu'))
# Второй слой подвыборки
model.add(MaxPooling2D(pool_size = (2, 2), dim_ordering="tf"))
# Слой регуляризации
model.add(Dropout(0.25))

# КЛАССИФИКАТОР
# Преобразование из двумерного вида в плоский
model.add(Flatten())
# Полносвязный слой
model.add(Dense(512, activation = 'relu'))
# Слой регуляризации
model.add(Dropout(0.5))
# Выходной слой
model.add(Dense(4, activation = 'softmax'))

# КОМПИЛЯЦИЯ СЕТИ
model.compile(loss = 'categorical_crossentropy',
             optimizer = 'SGD',
             metrics = ['accuracy'])

# ОБУЧЕНИЕ СЕТИ
model.fit(train_data, train_target,
         batch_size = 32,
         nb_epoch = 25,
         validation_split = 0.1,
         shuffle = True)